In [6]:

%load_ext autoreload
%autoreload 2

from agents.DotGobblers.config import PROJ_ROOT
from pathlib import Path
from GameAnalysis import GameAnalysis

#JSON files
balancedVSbalanced_json = Path(PROJ_ROOT, "www", "contest_balancedVSbalanced", "scores", "match_0.json")
balancedVSdefensive_json = Path(PROJ_ROOT, "www", "contest_balancedVSdefensive", "scores", "match_0.json")
balancedVSoffensive_json = Path(PROJ_ROOT, "www", "contest_balancedVSoffensive", "scores", "match_0.json")

offensiveVSoffensive_json = Path(PROJ_ROOT, "www", "contest_offensiveVSoffensive", "scores", "match_0.json")
offensiveVSdefensive_json = Path(PROJ_ROOT, "www", "contest_offensiveVSdefensive", "scores", "match_0.json")

defensiveVSdefensive_json = Path(PROJ_ROOT, "www", "contest_defensiveVSdefensive", "scores", "match_0.json")

#GameAnalysis objects
balancedVSbalanced_exp = GameAnalysis(balancedVSbalanced_json)
balancedVSdefensive_exp = GameAnalysis(balancedVSdefensive_json)
balancedVSoffensive_exp = GameAnalysis(balancedVSoffensive_json)

offensiveVSoffensive_exp = GameAnalysis(offensiveVSoffensive_json)
offensiveVSdefensive_exp = GameAnalysis(offensiveVSdefensive_json)

defensiveVSdefensive_exp = GameAnalysis(defensiveVSdefensive_json)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
balancedVSoffensive_exp.team_stats_dataframe

,points_percentage,points,wins,draws,losses,errors,sum_score,team
BlueBalanced,56.476190,1186.0,356.0,118.0,226.0,0.0,9737.0,BlueBalanced
RedOffensive,37.904762,796.0,226.0,118.0,356.0,0.0,6173.0,RedOffensive


In [12]:
balancedVSoffensive_exp.games_dataframe

,team_1,team_2,layout,score,winner,time_taken,match_number
0,RedOffensive,BlueBalanced,RANDOM81154050,28,RedOffensive,491.0,0
1,RedOffensive,BlueBalanced,RANDOM89728958,28,RedOffensive,491.0,0
2,RedOffensive,BlueBalanced,RANDOM62863058,10,BlueBalanced,491.0,0
3,RedOffensive,BlueBalanced,RANDOM39145404,28,BlueBalanced,491.0,0
4,RedOffensive,BlueBalanced,RANDOM46583831,28,BlueBalanced,491.0,0
...,...,...,...,...,...,...,...
695,RedOffensive,BlueBalanced,RANDOM20826224,28,BlueBalanced,491.0,0
696,RedOffensive,BlueBalanced,RANDOM92860334,28,RedOffensive,491.0,0
697,RedOffensive,BlueBalanced,RANDOM77923529,28,BlueBalanced,491.0,0
698,RedOffensive,BlueBalanced,RANDOM47412507,29,RedOffensive,491.0,0
